In [1]:
#Import required libraries
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import numpy as np
import nltk
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import datasets
from sklearn import metrics
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import BalancedRandomForestClassifier

/Users/troycassels/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
cd /Users/troycassels/Documents

/Users/troycassels/Documents


In [3]:
file_path = "Documents"

In [4]:
df = pd.read_csv(Path("tweets.csv"))

/var/folders/ds/yr__0mv52qg383xz2bm8ycbw0000gn/T/ipykernel_65225/240215716.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(Path("tweets.csv"))


In [5]:
df = df.dropna()

In [6]:
df = df[df["text"].str.contains("ChatGPT | chatgpt | #chatGPT | #chatgpt")==True]

In [7]:
df.head()

,user_name,text,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,hashtags,source
0,Math_Maniac,ChatGPT does ascii art\n#ChatGPT https://t.co/...,"Decatur, GA",I work with a number of tools. Right now it's ...,2020-07-20 14:24:50+00:00,467.0,1233.0,49216,False,2023-04-02 13:44:07+00:00,['ChatGPT'],Twitter Web App
1,SmashKoders,#ChatGPT is funny sometimes 😅 https://t.co/cSa...,Mumbai,My life changed while debugging code 😊,2022-06-18 04:34:18+00:00,8.0,126.0,239,False,2023-04-02 13:43:46+00:00,['ChatGPT'],Twitter Web App
2,Andreas Illenseer,Let’s put it in graphic terms. Drawing a #Unic...,"Mainz, Deutschland",Specialized in Regulatory & Digital Intelligen...,2017-10-31 19:35:24+00:00,63.0,188.0,88,False,2023-04-02 13:43:36+00:00,['Unicorn'],Twitter for iPhone
9,Myindianow,ChatGPT will not replace people. But people wh...,"Pune, India",🇮🇳,2022-03-02 03:02:43+00:00,14.0,33.0,52,False,2023-04-02 13:40:23+00:00,"['ChatGPT', 'OpenAI', 'AIart']",Twitter for Android
10,Rob,Italy banning open source AI.. #ChatGPT https:...,"London, UK",tech for @theBHF 🚀 ❤️,2009-04-25 15:16:05+00:00,207.0,754.0,1124,False,2023-04-02 13:40:12+00:00,['ChatGPT'],Twitter for iPhone


In [8]:
df.iloc[0]

user_name                                                 Math_Maniac
text                ChatGPT does ascii art\n#ChatGPT https://t.co/...
user_location                                             Decatur, GA
user_description    I work with a number of tools. Right now it's ...
user_created                                2020-07-20 14:24:50+00:00
user_followers                                                  467.0
user_friends                                                   1233.0
user_favourites                                                 49216
user_verified                                                   False
date                                        2023-04-02 13:44:07+00:00
hashtags                                                  ['ChatGPT']
source                                                Twitter Web App
Name: 0, dtype: object

In [9]:
df["user_created"] =df["user_created"].apply(lambda x: pd.Timestamp(x).timestamp())

In [10]:
df["date"] =df["date"].apply(lambda x: pd.Timestamp(x).timestamp())

In [11]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia= SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/troycassels/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [12]:
df['scores']=df['text'].apply(lambda body: sia.polarity_scores(str(body)))

In [13]:
def vader_scores(text, category):
    return vader.polarity_scores(text).get(category)

vader = SentimentIntensityAnalyzer()

df["Negative_Score"] = df.apply(lambda row : vader_scores(df["text"][row.name], "neg"),axis=1)
df["Neutral_Score"] = df.apply(lambda row : vader_scores(df["text"][row.name], "neu"),axis=1)
df["Positive_Score"] = df.apply(lambda row : vader_scores(df["text"][row.name], "pos"),axis=1)
df["Compound_Score"] = df.apply(lambda row : vader_scores(df["text"][row.name], "compound"),axis=1)


In [14]:
df["positive"] = np.where(df["Compound_Score"] >= 0.05 , 1, 0)

In [15]:
df["negative"] = np.where(df["Compound_Score"] <= - 0.05 , 1, 0)

In [16]:
df["neutral"] = np.where((df["positive"] + df["negative"] == 0), 1, 0)

In [17]:
df = df[df.neutral != 1]

In [18]:
df.head()

,user_name,text,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,hashtags,source,scores,Negative_Score,Neutral_Score,Positive_Score,Compound_Score,positive,negative,neutral
1,SmashKoders,#ChatGPT is funny sometimes 😅 https://t.co/cSa...,Mumbai,My life changed while debugging code 😊,1.655527e+09,8.0,126.0,239,False,1.680443e+09,['ChatGPT'],Twitter Web App,"{'neg': 0.0, 'neu': 0.58, 'pos': 0.42, 'compou...",0.000,0.580,0.420,0.4404,1,0,0
2,Andreas Illenseer,Let’s put it in graphic terms. Drawing a #Unic...,"Mainz, Deutschland",Specialized in Regulatory & Digital Intelligen...,1.509479e+09,63.0,188.0,88,False,1.680443e+09,['Unicorn'],Twitter for iPhone,"{'neg': 0.0, 'neu': 0.857, 'pos': 0.143, 'comp...",0.000,0.857,0.143,0.7574,1,0,0
11,VALIPOKKANN • 𝑵𝑭𝑻.𝑵𝒀𝑪 𝟐𝟑,"Good morning, world! Starting *chatgpt stuck h...",VALIPOKKANN.ETH.co,|| avant garde 🌏 கலைஞர் || new account 🙃||,1.659953e+09,733.0,369.0,31198,False,1.680443e+09,['ChatGPT'],Twitter Web App,"{'neg': 0.201, 'neu': 0.683, 'pos': 0.117, 'co...",0.201,0.683,0.117,-0.2714,0,1,0
12,Anthony Hadfield Ph.D.,#GPT4 #ChatGPT #OpenAI #ArtificialIntelligence...,"St Petersburg, FL","I have studied CompSci, AI, RL, LA, Vision, Py...",1.457680e+09,26.0,15.0,291,False,1.680443e+09,"['GPT4', 'ChatGPT', 'OpenAI', 'ArtificialIntel...",Twitter Web App,"{'neg': 0.157, 'neu': 0.637, 'pos': 0.205, 'co...",0.157,0.637,0.205,0.5041,1,0,0
13,Ines Hegedus-Garcia,Join us at Leveraging Generative AI &amp; Chat...,Miami,Architectural REALTOR at Avanti Way Realty pas...,1.175280e+09,14174.0,3351.0,1564,False,1.680443e+09,"['AI', 'ChatGPT']",Twitter for iPhone,"{'neg': 0.0, 'neu': 0.855, 'pos': 0.145, 'comp...",0.000,0.855,0.145,0.2960,1,0,0


In [19]:
df.iloc[0]

user_name                                                 SmashKoders
text                #ChatGPT is funny sometimes 😅 https://t.co/cSa...
user_location                                                  Mumbai
user_description               My life changed while debugging code 😊
user_created                                             1655526858.0
user_followers                                                    8.0
user_friends                                                    126.0
user_favourites                                                   239
user_verified                                                   False
date                                                     1680443026.0
hashtags                                                  ['ChatGPT']
source                                                Twitter Web App
scores              {'neg': 0.0, 'neu': 0.58, 'pos': 0.42, 'compou...
Negative_Score                                                    0.0
Neutral_Score       

In [20]:
df.dtypes

user_name            object
text                 object
user_location        object
user_description     object
user_created        float64
user_followers       object
user_friends         object
user_favourites      object
user_verified        object
date                float64
hashtags             object
source               object
scores               object
Negative_Score      float64
Neutral_Score       float64
Positive_Score      float64
Compound_Score      float64
positive              int64
negative              int64
neutral               int64
dtype: object

In [21]:
X_not_categorical = df[["user_created","user_followers","user_friends","user_favourites"]]
X_not_categorical.head()

,user_created,user_followers,user_friends,user_favourites
1,1.655527e+09,8.0,126.0,239
2,1.509479e+09,63.0,188.0,88
11,1.659953e+09,733.0,369.0,31198
12,1.457680e+09,26.0,15.0,291
13,1.175280e+09,14174.0,3351.0,1564


In [22]:
X_categorical_exc_location = df[["user_verified","source"]]
X_categorical_exc_location.head()

,user_verified,source
1,False,Twitter Web App
2,False,Twitter for iPhone
11,False,Twitter Web App
12,False,Twitter Web App
13,False,Twitter for iPhone


In [23]:
X_categorical_exc_location = pd.get_dummies(X_categorical_exc_location)
X_categorical_exc_location.head()

,user_verified_False,user_verified_True,user_verified_False,user_verified_True,source_AI Assistant,source_AI In Plain English,source_AI Social Post,source_ANews Twitter,source_AYappTesting,source_Admin Kuvira Crypto Bot,...,source_tweetmonk.com,source_twefed,source_twitter-app503,source_twitterDemoTest3,source_twitterLisaHunter,source_twitter_com_flume,source_twitto_be_bot,source_usetwype,source_vassbots,source_vijay ds project
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
X = pd.concat([X_not_categorical, X_categorical_exc_location], axis=1)
X.head()

,user_created,user_followers,user_friends,user_favourites,user_verified_False,user_verified_True,user_verified_False,user_verified_True,source_AI Assistant,source_AI In Plain English,...,source_tweetmonk.com,source_twefed,source_twitter-app503,source_twitterDemoTest3,source_twitterLisaHunter,source_twitter_com_flume,source_twitto_be_bot,source_usetwype,source_vassbots,source_vijay ds project
1,1.655527e+09,8.0,126.0,239,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.509479e+09,63.0,188.0,88,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,1.659953e+09,733.0,369.0,31198,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,1.457680e+09,26.0,15.0,291,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,1.175280e+09,14174.0,3351.0,1564,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
y = df["positive"]

In [26]:
y.head()

1     1
2     1
11    0
12    1
13    1
Name: positive, dtype: int64

In [27]:
X.head()

,user_created,user_followers,user_friends,user_favourites,user_verified_False,user_verified_True,user_verified_False,user_verified_True,source_AI Assistant,source_AI In Plain English,...,source_tweetmonk.com,source_twefed,source_twitter-app503,source_twitterDemoTest3,source_twitterLisaHunter,source_twitter_com_flume,source_twitto_be_bot,source_usetwype,source_vassbots,source_vijay ds project
1,1.655527e+09,8.0,126.0,239,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.509479e+09,63.0,188.0,88,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,1.659953e+09,733.0,369.0,31198,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,1.457680e+09,26.0,15.0,291,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,1.175280e+09,14174.0,3351.0,1564,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# Split data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [29]:
# Instantiate a StandardScaler instance
scaler = StandardScaler()

# Fit the training data to the standard scaler
X_scaler = scaler.fit(X_train)

# Transform the training data using the scaler
X_train_scaled = X_scaler.transform(X_train)

# Transform the testing data using the scaler
X_test_scaled = X_scaler.transform(X_test)

In [30]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=1)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data
rf_predictions = rf_model.predict(X_test_scaled)

In [31]:
# Instantiate a BalancedRandomForestClassifier instance
brf = BalancedRandomForestClassifier()

# Fit the model to the training data
brf.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier()

In [32]:
# Predict labels for testing features
brf_predictions = brf.predict(X_test_scaled)

In [33]:
from sklearn.metrics import confusion_matrix

In [34]:
# Print the confusion matrix for balanced random forest data
confusion_matrix(y_test, brf_predictions)

array([[3118, 2083],
       [7644, 9836]])

In [35]:
print(classification_report_imbalanced(y_test, brf_predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.29      0.60      0.56      0.39      0.58      0.34      5201
          1       0.83      0.56      0.60      0.67      0.58      0.34     17480

avg / total       0.70      0.57      0.59      0.61      0.58      0.34     22681



In [36]:
abc = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1)

In [37]:
model = abc.fit(X_train, y_train)

In [38]:
y_pred = model.predict(X_test)

In [39]:
confusion_matrix(y_test, y_pred)

array([[    9,  5192],
       [    4, 17476]])

In [40]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.69      0.00      0.00      5201
           1       0.77      1.00      0.87     17480

    accuracy                           0.77     22681
   macro avg       0.73      0.50      0.44     22681
weighted avg       0.75      0.77      0.67     22681

